# Reading, Writing and Validating Data in PySpark HW

Welcome to your first coding homework assignment in PySpark! I hope you enjoyed the lecture on Reading, Writing and Validating dataframes. Now it's time to put what you've learned into action! 

I've included several instructions below to help guide you through this homework assignment which I hope will get you feeling even comfortable reading, writing and validating dataframes. If you get stuck at any point, feel free to jump to the next lecture where I will guide you through my solutions to the HW assignment. 

Have fun!

Let's dig right in!


## But first things first.....
We need to always begin every Spark session by creating a Spark instance. Let's go ahead and use the method we learned in the lecture in the cell below. Also see if you can remember how to open the Spark UI (using a link that automatically guides you there). 

In [8]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [58]:
pip install findspark

In [2]:
import pyspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

## Next let's start by reading a basic csv dataset

Download the pga_tour_historical dataset that is attached to this lecture and save it whatever folder you want, then read it in. 

**Data Source:** https://www.kaggle.com/bradklassen/pga-tour-20102018-data

Rememer to try letting Spark infer the header and infer the Schema types!

In [4]:
#First Method (using PySpark)
df = spark.read.csv('pga_tour_historical.csv')
df.show()

#Second Method (using pandas)
#import pandas as pd
#data = pd.read_csv('pga_tour_historical.csv')
#display(data)

+---------------+------+----------------+--------------------+-----+
|            _c0|   _c1|             _c2|                 _c3|  _c4|
+---------------+------+----------------+--------------------+-----+
|    Player Name|Season|       Statistic|            Variable|Value|
|Robert Garrigus|  2010|Driving Distance|Driving Distance ...|   71|
|   Bubba Watson|  2010|Driving Distance|Driving Distance ...|   77|
| Dustin Johnson|  2010|Driving Distance|Driving Distance ...|   83|
|Brett Wetterich|  2010|Driving Distance|Driving Distance ...|   54|
|    J.B. Holmes|  2010|Driving Distance|Driving Distance ...|  100|
|      John Daly|  2010|Driving Distance|Driving Distance ...|   63|
|  Graham DeLaet|  2010|Driving Distance|Driving Distance ...|   88|
|  Angel Cabrera|  2010|Driving Distance|Driving Distance ...|   64|
| Charles Warren|  2010|Driving Distance|Driving Distance ...|   64|
|    D.J. Trahan|  2010|Driving Distance|Driving Distance ...|   92|
|  Martin Flores|  2010|Driving Di

## 1. View first 5 lines of dataframe
First generate a view of the first 5 lines of the dataframe to get an idea of what is inside. We went over two ways of doing this... see if you can remember BOTH ways. 

In [5]:
#First Method
df.show(5)

#Second Method
# df = pd.DataFrame(data)
# df.head(5)

#Third Method (learning in progress)
# from pyspark import SparkContext
# from pyspark.sql import SQLContext
# adf = SQLContext.read.format('csv')

+---------------+------+----------------+--------------------+-----+
|            _c0|   _c1|             _c2|                 _c3|  _c4|
+---------------+------+----------------+--------------------+-----+
|    Player Name|Season|       Statistic|            Variable|Value|
|Robert Garrigus|  2010|Driving Distance|Driving Distance ...|   71|
|   Bubba Watson|  2010|Driving Distance|Driving Distance ...|   77|
| Dustin Johnson|  2010|Driving Distance|Driving Distance ...|   83|
|Brett Wetterich|  2010|Driving Distance|Driving Distance ...|   54|
+---------------+------+----------------+--------------------+-----+
only showing top 5 rows



## 2. Print the schema details

Now print the details of the dataframes schema that Spark infered to ensure that it was infered correctly. Sometimes it is not infered correctly, so we need to watch out!

In [6]:
schema_details = df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)



## 3. Edit the schema during the read in

We can see from the output above that Spark did not correctly infer that the "value" column was an integer value. Let's try specifying the schema this time to let spark know what the schema should be.

Here is a link to see a list of PySpark data types in case you need it (also attached to the lecture): 
https://spark.apache.org/docs/latest/sql-ref-datatypes.html

In [7]:
from pyspark.sql.functions import *

data_type_map = {
    '_c1': 'int',
    '_c4': 'double'
} 

revamped_df = df.select([col(column_schema[0]).cast(data_type_map.get(column_schema[0], column_schema[1])) for column_schema in df.dtypes])

revamped_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: double (nullable = true)



## 4. Generate summary statistics for only one variable

See if you can generate summary statistics for only the "Value" column using the .describe function

(count, mean, stddev, min, max) 

In [8]:
df.describe('_c4').show()

+-------+------------------+
|summary|               _c4|
+-------+------------------+
|  count|           2696906|
|   mean|137053.96543856148|
| stddev| 6046342.246458573|
|    min|        $1,001,580|
|    max|   the Memorial/Mu|
+-------+------------------+



## 5. Generate summary statistics for TWO variables
Now try to generate ONLY the count min and max for BOTH the "Value" and "Season" variable using the select. You can't use the .describe function for this one but see if you can remember which function you CAN use. 

In [9]:
#df.describe('_c1', '_c4').show() == df.select('_c1', '_c4').describe().show() 

cau5 = df.select('_c1', '_c4').summary('min', 'max').show()

+-------+------+---------------+
|summary|   _c1|            _c4|
+-------+------+---------------+
|    min|  2010|     $1,001,580|
|    max|Season|the Memorial/Mu|
+-------+------+---------------+



## 6. Write a parquet file

Now try writing a parquet file (not partitioned) from the pga dataset. But first create a new dataframe containing ONLY the the "Season" and "Value" fields (using the "select command you used in the question above) and write a parquet file partitioned by "Season". This is a bit of a challenge aimed at getting you ready for material that will be covered later on in the course. Don't feel bad if you can't figure it out.

*Note that if any of your variable names contain spaces, spark will produce an error message with this call. That is why we are selecting ONLY the "Season" and "Value" fields. Ideally we should renamed those columns but we haven't gotten to that yet in this course but we will soon!*

In [47]:
temp_df = revamped_df.select('_c1', '_c4')

In [48]:
#temp_df.write.parquet('tour') Doesn't work on this laptop

#Another approach:
import pandas as pd
pandas_df = temp_df.toPandas() 
pandas_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2740404 entries, 0 to 2740403
Data columns (total 2 columns):
 #   Column  Dtype  
---  ------  -----  
 0   _c1     float64
 1   _c4     float64
dtypes: float64(2)
memory usage: 41.8 MB


In [50]:
# display(pandas_df)
pandas_df.to_parquet('tour.parquet')

## 7. Write a partioned parquet file

You will need to use the same limited dataframe that you created in the previous question to accomplish this task as well. 

In [77]:
#df1.write.partitionBy("_c1").mode("overwrite").parquet("tour2.parquet")

pandas_df2 = pandas_df.to_parquet('partioned.parquet', partition_cols = '_c1')

## 8. Read in a partitioned parquet file

Now try reading in the partitioned parquet file you just created above. 

In [116]:
#df1.read.parquet("tour2.parquet")

pd.read_parquet('partioned.parquet', engine='auto')

""


## 9. Reading in a set of paritioned parquet files

Now try only reading Seasons 2010, 2011 and 2012.

In [102]:
pd.read_parquet('partioned.parquet/_c1=2011')

,_c4
0,57.000
1,85.000
2,72.000
3,83.000
4,93.000
...,...
305107,0.087
305108,0.128
305109,0.101
305110,0.224


In [104]:
pd.read_parquet('partioned.parquet/_c1=2012')

,_c4
0,68.000
1,61.000
2,89.000
3,70.000
4,60.000
...,...
311653,0.211
311654,0.177
311655,0.034
311656,0.304


## 10. Create your own dataframe

Try creating your own dataframe below using PySparks *.createDataFrame* function. See if you can make one that contains 4 variables and at least 3 rows. 

Let's see how creative you can get on the content of the dataframe :)

In [4]:
new_spark = SparkSession.builder.getOrCreate()
data =[("1 ","John Smith","36","M"),
              ("2","Cindy James","28","F"),
              ("3","Mike Douglas","33","M")]
columns=["ID","Name","Age","Sex"]
df3=new_spark.createDataFrame(data,columns)
df3.show()

+---+------------+---+---+
| ID|        Name|Age|Sex|
+---+------------+---+---+
| 1 |  John Smith| 36|  M|
|  2| Cindy James| 28|  F|
|  3|Mike Douglas| 33|  M|
+---+------------+---+---+



## We're done! Great job!